# Fully Connected Deep Neural Network
## 预处理

In [1]:
from load import load_all_dataset
X_train, y_train, X_test, y_test = load_all_dataset()
import numpy as np
np.set_printoptions(edgeitems=5,
                    linewidth=1000,
                    formatter={"float":lambda x: "{:.3f}".format(x)})

Train data
Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples
 Optical Dataset labels composed of
46110 labels of source samples
438 labels of target samples

Test data
Optical Dataset composed of
0 source samples
0 source background samples
17758 target labeled samples
0 target unlabeled samples
47275 target background samples
 Optical Dataset labels composed of
0 labels of source samples
17758 labels of target samples



In [2]:
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
# 去除NaN
class FeatureExtractor:
    def transform(self, X):
        '''
        Parameters
        ----------
        `X`: ndarray of (sample, 672, 10)
            3D input dataset(sample, time, features)
        
        Returns
        -------
        `X`: ndarray of (sample, 6720)
            The filtered dataset
        '''
        np.nan_to_num(X, copy=False)
        X = X.reshape(X.shape[0], -1)
        return X

fe = FeatureExtractor()
X_train.source = fe.transform(X_train.source)
X_train.target = fe.transform(X_train.target)
X_test.target = fe.transform(X_test.target)
print("X_train.source:", X_train.source.shape)
print("X_train.target:", X_train.target.shape)
print("X_test.target:", X_test.target.shape)

X_train.source: (46110, 6720)
X_train.target: (438, 6720)
X_test.target: (17758, 6720)


In [4]:
# 将数据集转换为TensorFlow格式
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.source, y_train.source)).batch(16)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_train.target, y_train.target)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.target, y_test.target))

# 额外操作
# train_dataset = train_dataset.map( lambda x, y: (tf.image.random_flip_left_right(x), y) ) # array must be 3D
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

## 搭建网络模型
参考资料

1. [Build your first Neural Network in TensorFlow 2](https://towardsdatascience.com/building-your-first-neural-network-in-tensorflow-2-tensorflow-for-hackers-part-i-e1e2f1dfe7a0)

In [5]:
# Sequential groups a linear stack of layers into a tf.keras.Model.
# https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
model = tf.keras.Sequential()
model.add( layers.Flatten(input_shape=(6720,), name="Input_Layer") )

num_fully_connected_layers = 10
for i in range(num_fully_connected_layers):
    model.add( layers.Dense(256, activation="relu", name="Layer{}".format(i+1)) )

model.add( layers.Dropout(0.5, name="Layer-1"))
model.add( layers.Dense(1, activation='sigmoid', name="Output_Layer") )

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['acc'])
# model.summary()

## 训练模型
- if 'softmax' in the last layer, output is 0 or 1
    - 'categorical_crossentropy' returns NaN, 'binary_crossentropy' acc ~ 0.1
- if `'sigmoid'` in the last layer, output is the probability of 1
    - 'categorical_crossentropy' returns NaN, `'binary_crossentropy'` acc ~ 0.8 to 0.9

In [6]:
import datetime as dt
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./log/{}'.format(
      dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S DNN")), write_images=True),
  ]
model.fit(train_dataset, epochs=10, steps_per_epoch=20,
          validation_data=valid_dataset, validation_steps=3,
          # callbacks=callbacks
          )

Epoch 1/10
20/20 [==============================] - 1s 17ms/step - loss: 96155.5234 - acc: 0.7156 - val_loss: 125621.5938 - val_acc: 0.7083
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 28032.3027 - acc: 0.8125 - val_loss: 32221.4766 - val_acc: 0.7083
Epoch 3/10
20/20 [==============================] - 0s 7ms/step - loss: 15512.1777 - acc: 0.7375 - val_loss: 24648.7090 - val_acc: 0.7083
Epoch 4/10
20/20 [==============================] - 0s 7ms/step - loss: 24440.2363 - acc: 0.7375 - val_loss: 2642.0518 - val_acc: 0.6458
Epoch 5/10
20/20 [==============================] - 0s 6ms/step - loss: 39479.8984 - acc: 0.6531 - val_loss: 7003.0039 - val_acc: 0.6458
Epoch 6/10
20/20 [==============================] - 0s 6ms/step - loss: 22246.1367 - acc: 0.7594 - val_loss: 79082.9531 - val_acc: 0.7083
Epoch 7/10
20/20 [==============================] - 0s 6ms/step - loss: 8470.0879 - acc: 0.7750 - val_loss: 13881.5068 - val_acc: 0.7083
Epoch 8/10
20/20 [=================

## 预测概率

In [7]:
print("X_test.target.shape:", X_test.target.shape)
# X = X_test.target.reshape(X_test.target.shape[0], -1)
# print(X.shape)
y_pred = model.predict(X_test.target).transpose()
print("Predicted:", y_pred, y_pred.shape)
print("True:      ", y_test.target, y_test.target.shape)

X_test.target.shape: (17758, 6720)
Predicted: [[0.006 0.006 0.005 0.006 0.006 ... 0.019 0.099 0.025 0.011 0.370]] (1, 17758)
True:       [0.000 0.000 0.000 0.000 0.000 ... 0.000 1.000 0.000 0.000 1.000] (17758,)


## 查看Tensorboard

In [8]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.
